### Reddit PRAW testing

In [1]:
import praw
import pandas as pd
from pydantic_settings import BaseSettings, SettingsConfigDict
from loguru import logger


In [3]:
class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_prefix="reddit_")
    api_id : str
    api_secret: str
    username: str
    password: str

In [5]:
cfg=Settings()

In [6]:
reddit = praw.Reddit(user_agent=True,
                        client_id=cfg.api_id,
                        client_secret=cfg.api_secret,
                        username=cfg.username,
                        password=cfg.password)

In [18]:
url="https://www.reddit.com/r/migraine/comments/12gvvx3/what_headachemigraine_meds_is_most_favorable_in/"
submission=reddit.submission(url=url)
submission.comments.replace_more(limit=0)

[]

In [19]:
post_data = {
    "title": submission.title,
    "score": submission.score,
    "url": submission.url,
    "subreddit": str(submission.subreddit),
    "author": str(submission.author),
    "created_utc": submission.created_utc,
    "text": submission.selftext,
    "comments": [{
        "author": str(comment.author),
        "body": comment.body,
        "score": comment.score,
        "created_utc": comment.created_utc
    } for comment in submission.comments]
}

In [20]:
len(post_data["comments"])

76

In [ ]:
cfg = Settings()

class Scraper():
    def __init__(self,
                client_id: str = cfg.api_id,
                client_secret: str = cfg.api_secret,
                username: str = cfg.username,
                password: str = cfg.password
                ):
        self.reddit = praw.Reddit(user_agent=True,
                                    client_id=client_id,
                                    client_secret=client_secret,
                                    username=username,
                                    password=password)
        logger.info("Reddit API successfully initialized.")
    
    def _scrape_reddit_post(self, url: str):
        submission = self.reddit.submission(url=url)
        submission.comments.replace_more(limit=0)

        post_data = {
            "title": submission.title,
            "score": submission.score,
            "url": submission.url,
            "subreddit": str(submission.subreddit),
            "author": str(submission.author),
            "created_utc": submission.created_utc,
            "text": submission.selftext,
            "comments": [{
                "author": str(comment.author),
                "body": comment.body,
                "score": comment.score,
                "created_utc": comment.created_utc
            } for comment in submission.comments]
        }
        
        return post_data
